In [ ]:
import pickle
import torch
import numpy as np
save_path = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/e4e/experiments_default_lr/inversions/"

In [ ]:
latents = torch.load(f"{save_path}latents.pt")
with open(f"{save_path}file_paths.pkl", 'rb') as f:
    file_paths = pickle.load(f)

latents_dict = {}
for i, file in enumerate(file_paths):
    sku = file.split('/')[-1].split('_')[-1].split('.')[0]
    latents_dict[sku] = latents[i]

In [ ]:
# save file paths
with open(f"{save_path}latents_dict.pkl", 'wb') as handle:
    pickle.dump(latents_dict, handle)

In [ ]:
# Initalize original custom SG2-Ada generator
os.chdir("../../stylegan2-ada-pytorch/")
model_path = "/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Models/Stylegan2_Ada/Experiments/00003-stylegan2_ada_images-mirror-auto2-kimg1000-resumeffhq512/network-snapshot-000920.pkl"
with open(model_path, 'rb') as f:
    architecture = pickle.load(f)
    G = architecture['G_ema'].to('cuda')  # torch.nn.Module 
    D = architecture['D'].to('cuda')

### Test some images

In [ ]:
from PIL import Image
sample = np.random.choice(list(latents_dict.keys()),1)[0]
orig_image = Image.open(f"/pfs/work7/workspace/scratch/tu_zxmav84-thesis/Data.nosync/Zalando_Germany_Dataset/dresses/images/square_images/{sample}.jpg")
latent = latents_dict[sample]
img = G.synthesis(latent.to('cuda'))
img_perm = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
img_sg2 = Image.fromarray(img_perm[0].cpu().numpy(), 'RGB')

display(orig_image)
display(img_sg2)
